In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from natsort import natsorted, ns
import cv2
import dlib
import os
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold

import math

In [ ]:
# Create list of image names and corresponding emotion classifications
image_dic = pd.read_excel('../Datasets/labels_A.xlsx')
image_dic = image_dic[['img_name.jpg', 'smiling']] # Choose columns which are of importance
df = pd.DataFrame(image_dic)
df.to_excel('../Datasets/source_emotions/labels_A2.xlsx',index=False)

In [ ]:
# Separate smiling and not_smiling images and corresponding labels into folders

source_emotions = pd.read_excel('../Datasets/source_emotions/labels_A2.xlsx')
source_images_file_paths = glob.glob ("../Datasets/source_images/*.jpg") #find all paths which match the given path
source_images_file_paths = natsorted(source_images_file_paths) #sort the list of file names such that the image list will be in the correct order

smiling_images = []
not_smiling_images = []

smiling_directory = "../Datasets/sorted_sets/smiling/"
not_smiling_directory = "../Datasets/sorted_sets/not_smiling/"

for file_path in source_images_file_paths:
    image = cv2.imread(file_path, cv2.COLOR_RGB2BGR) #read the image
    image_name = os.path.basename(file_path)
    image_label = source_emotions[source_emotions['img_name.jpg']==image_name]['smiling'].iloc[0]
    if(image_label == 1):
        smiling_images.append(image)
        directory = ''.join([smiling_directory,os.path.basename(image_name)])
        cv2.imwrite(directory, image)
    else:
        not_smiling_images.append(image)
        directory = ''.join([not_smiling_directory,os.path.basename(image_name)])
        cv2.imwrite(directory, image)


In [ ]:
# Deetcting faces and cropping

faceDet = cv2.CascadeClassifier("../OpenCV_FaceCascade/haarcascade_frontalface_default.xml")
faceDet_two = cv2.CascadeClassifier("../OpenCV_FaceCascade/haarcascade_frontalface_alt2.xml")
faceDet_three = cv2.CascadeClassifier("../OpenCV_FaceCascade/haarcascade_frontalface_alt.xml")
faceDet_four = cv2.CascadeClassifier("../OpenCV_FaceCascade/haarcascade_frontalface_alt_tree.xml")
emotions = ["smiling", "not_smiling"] #Define emotions

net = cv2.dnn.readNetFromCaffe("../deep-learning-face-detection/deploy.prototxt.txt", "../deep-learning-face-detection/res10_300x300_ssd_iter_140000.caffemodel")

def DNN_Face_Detection(emotion):
    files = glob.glob("../Datasets/sorted_sets/%s/*.jpg" %emotion) #Get list of all images with emotion
    filenumber = 0
    for f in files:
        # load the input image and construct an input blob for the image
        # by resizing to a fixed 300x300 pixels and then normalizing it
        image = cv2.imread(f)
        (h, w) = image.shape[:2]
        #blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        
        BGR_ave = image.mean(axis=(0,1))
        blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (BGR_ave[2], BGR_ave[1], BGR_ave[0]))
        net.setInput(blob)
        detections = net.forward()

        actual_detections = detections[0][0]
        detection_accuracies = detections[0][0][:,2]

        index_values = np.argsort(detection_accuracies)
        index_values = index_values[::-1]

        sorted_detections = [actual_detections[i] for i in index_values]
        sorted_detections = np.asarray(sorted_detections)

        confidence = sorted_detections[0, 2]

        # filter out weak detections by ensuring the `confidence` is greater than the minimum confidence
        if confidence > 0.9:
            # compute the (x, y)-coordinates of the bounding box for the object
            box = sorted_detections[0, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            cropped = image[startY:endY, startX:endX] #Cut the frame to size
            try:
                out = cv2.resize(cropped, (300, 300)) #Resize face so all images have same size
                cv2.imwrite("../Datasets/A2_dataset_DNN/%s/%s.jpg" %(emotion, filenumber), out) #Write image - don' need to worry about keeping track of labels associated because already sorted
                filenumber += 1 #Increment image number
            except:
                pass #If error, pass file
                    
def detect_face_DNN(emotion):
    files = glob.glob("../Datasets/sorted_sets/%s/*.jpg" %emotion) #Get list of all images with emotion
    filenumber = 0
    for f in files:
        # load the input image and construct an input blob for the image
        # by resizing to a fixed 300x300 pixels and then normalizing it
        image = cv2.imread(f)
        (h, w) = image.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        print(detections.shape[2])
        # loop over the detections
        for i in range(0, detections.shape[2]):
            # extract the confidence (i.e., probability) associated with the
            # prediction
            confidence = detections[0, 0, i, 2]

            # filter out weak detections by ensuring the `confidence` is greater than the minimum confidence
            if confidence > 0.999:
                # compute the (x, y)-coordinates of the bounding box for the object
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                cropped = image[startY:endY, startX:endX] #Cut the frame to size
                try:
                    out = cv2.resize(cropped, (300, 300)) #Resize face so all images have same size
                    cv2.imwrite("../Datasets/A2_dataset_DNN/%s/%s.jpg" %(emotion, filenumber), out) #Write image - don' need to worry about keeping track of labels associated because already sorted
                    filenumber += 1 #Increment image number
                except:
                    pass #If error, pass file

def detect_faces(emotion):
    files = glob.glob("../Datasets/sorted_sets/%s/*.jpg" %emotion) #Get list of all images with emotion
    filenumber = 0
    for f in files:
        frame = cv2.imread(f) #Open image
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #Convert image to grayscale
        #Detect face using 4 different classifiers
        face = faceDet.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face_two = faceDet_two.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face_three = faceDet_three.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face_four = faceDet_four.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        #Go over detected faces, stop at first detected face, return empty if no face.
        if len(face) == 1:
            facefeatures = face
        elif len(face_two) == 1:
            facefeatures = face_two
        elif len(face_three) == 1:
            facefeatures = face_three
        elif len(face_four) == 1:
            facefeatures = face_four
        else:
            facefeatures = ""
        #Cut and save face
        for (x, y, w, h) in facefeatures: #get coordinates and size of rectangle containing face
            #print ("face found in file: %s" %f)
            gray = gray[y:y+h, x:x+w] #Cut the frame to size
            try:
                out = cv2.resize(gray, (350, 350)) #Resize face so all images have same size
                cv2.imwrite("../Datasets/A2_dataset/%s/%s.jpg" %(emotion, filenumber), out) #Write image - don' need to worry about keeping track of labels associated because already sorted
            except:
                pass #If error, pass file
        filenumber += 1 #Increment image number
for emotion in emotions:
    DNN_Face_Detection(emotion) #Call functiona

In [ ]:
net = cv2.dnn.readNetFromCaffe("../deep-learning-face-detection/deploy.prototxt.txt", "../deep-learning-face-detection/res10_300x300_ssd_iter_140000.caffemodel")

def face_rect_DNN(image):
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()
    
    actual_detections = detections[0][0]
    detection_accuracies = detections[0][0][:,2]
    
    index_values = np.argsort(detection_accuracies)
    index_values = index_values[::-1]

    sorted_detections = [actual_detections[i] for i in index_values]
    sorted_detections = np.asarray(sorted_detections)

    confidence = sorted_detections[0, 2]

    # filter out weak detections by ensuring the `confidence` is
    # greater than the minimum confidence
    if confidence > 0.95:
        # compute the (x, y)-coordinates of the bounding box for the
        # object
        box = sorted_detections[0, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        faceBoxRectangleS = dlib.rectangle(left=startX, top=startY, right=endX, bottom=endY)
        return faceBoxRectangleS
    return []

In [ ]:
# Emotion classisification using facial landmarks

emotions = ["smiling", "not_smiling"] #Define emotions

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") #Or set this to whatever you named the downloaded file

clf1 = SVC(kernel='rbf', gamma= 0.001, C= 10, decision_function_shape= 'OVO' ,probability=True)#, verbose = True) #Set the classifier as a support vector machine
clf2 = SVC(kernel='rbf', gamma= 'scale', C = 10, decision_function_shape= 'OVO' ,probability=True)#, verbose = True) #Set the classifier as a support vector machine

data = {} #Make dictionary for all values

#data['landmarks_vectorised'] = []
def get_files(emotion): #Define function to get file list, randomly shuffle it and split 80/20
    files = glob.glob("../Datasets/A2_dataset_DNN/%s/*.jpg" %emotion)
    random.shuffle(files)
    training = files[:int(len(files)*0.8)] #get first 80% of file list
    prediction = files[-int(len(files)*0.2):] #get last 20% of file list
    return training, prediction

def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10,100,1000]
    gammas = [0.00001, 0.0001, 0.001, 0.01, 0.1,1,10,'scale']
    decision_function_shapes = ['OVO', 'OVA']
    param_grid = {'C': Cs, 'gamma' : gammas, 'decision_function_shape': decision_function_shapes}
    grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    best_params = grid_search.best_params_
    mean_CV_score = grid_search.cv_results_['mean_test_score']
    return best_params, mean_CV_score

def get_landmarks(image):
    detections = detector(image, 1)
    for k,d in enumerate(detections): #For all detected face instances individually
        shape = predictor(image, d) #Draw Facial Landmarks with the predictor class
        xlist = []
        ylist = []
        for i in range(1,68): #Store X and Y coordinates in two lists
            xlist.append(float(shape.part(i).x))
            ylist.append(float(shape.part(i).y))
        xmean = np.mean(xlist)
        ymean = np.mean(ylist)
        xcentral = [(x-xmean) for x in xlist]
        ycentral = [(y-ymean) for y in ylist]
        landmarks_vectorised = []
        for x, y, w, z in zip(xcentral, ycentral, xlist, ylist):
            landmarks_vectorised.append(w)
            landmarks_vectorised.append(z)
            meannp = np.asarray((ymean,xmean))
            coornp = np.asarray((z,w))
            dist = np.linalg.norm(coornp-meannp)
            landmarks_vectorised.append(dist)
            landmarks_vectorised.append((math.atan2(y, x)*360)/(2*math.pi))
        data['landmarks_vectorised'] = landmarks_vectorised
    if len(detections) < 1:
        data['landmarks_vectorised'] = "error"
        
def make_sets():
    training_data = []
    training_labels = []
    prediction_data = []
    prediction_labels = []
    for emotion in emotions:
        print(" working on %s" %emotion)
        training, prediction = get_files(emotion)
        #Append data to training and prediction list, and generate labels 0-1
        count = 1;
        for item in training:
            image = cv2.imread(item) #open image
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #convert to grayscale
            clahe_image = clahe.apply(gray)
            get_landmarks(gray)
            if data['landmarks_vectorised'] == "error":
                count +=1
            else:
                training_data.append(data['landmarks_vectorised']) #append image array to training data list
                training_labels.append(emotions.index(emotion))
        count = 1;
        for item in prediction:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            clahe_image = clahe.apply(gray)
            get_landmarks(gray)
            if data['landmarks_vectorised'] == "error":
                count +=1
            else:
                prediction_data.append(data['landmarks_vectorised'])
                prediction_labels.append(emotions.index(emotion))
    return training_data, training_labels, prediction_data, prediction_labels

print("Making set")
training_data, training_labels, prediction_data, prediction_labels = make_sets()
npar_train = np.array(training_data) #Turn the training set into a numpy array for the classifier
npar_trainlabs = np.array(training_labels)
npar_pred = np.array(prediction_data)

#best_params, mean_CV_score = svc_param_selection(npar_train,npar_trainlabs,5)
#print(best_params)
#plt.scatter(mean_CV_score)

print("training SVM") #train SVM
clf2.fit(npar_train, npar_trainlabs)
print("getting accuracy") #Use score() function to get accuracy
pred_lin = clf2.score(npar_pred, prediction_labels)
print ("Accuracy: ", pred_lin)

In [ ]:
best_params, mean_CV_score = svc_param_selection(npar_train,npar_trainlabs,5)
print(best_params)

In [ ]:
print(best_params)
print(mean_CV_score.shape)
print(mean_CV_score.reshape(mean_CV_score.shape[0],1))

In [ ]:
def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 10)):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    axes : array of 3 axes, optional (default=None)
        Axes to use for plotting the curves.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.title(title)
    plt.ylim(ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes)
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    print(train_sizes)
    # Plot learning curve
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    plt.legend(loc="best")

    return plt


X, y = npar_train, npar_trainlabs

title = "Learning Curves (SVM, RBF kernel, $\gamma=0.01, C = 10, decision function shape = OVO$)"

# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
# SVC is more expensive so we do a lower number of CV iterations:
kf = KFold(n_splits=5)

c= 10
gamma = 0.001
method = 'OVO'

estimator = SVC(kernel='rbf',C = c , gamma = gamma, decision_function_shape = method)
plt = plot_learning_curve(estimator, title, X, y, ylim=(0.75, 1.01),
                    cv=kf, n_jobs=4)
#plt.savefig('../plots_A1/10_learningcurve.png')
plt.show()